## Zonal Statistics

In this tutorial, you'll learn how to use [xarray-spatial](https://xarray-spatial.readthedocs.io/en/latest/index.html) to work with zonal statistics. Zonal statistics help you better understand data from one source by analyzing it for different zones defined by another source. This operation uses two datasets: One dataset, the *zones raster*, defines one or more zones. A second dataset, the *values raster*, contains the data you want to analyze for each of the zones defined by the first dataset. If you're familiar with SQL or dataframe libraries like [pandas](https://pandas.pydata.org/), this is similar to a group by operation: you essentially group the *values* raster by the *zones* raster.

In this tutorial, we'll use the [Esri / Observatory 10-Meter Land Cover](https://planetarycomputer.microsoft.com/dataset/io-lulc) dataset, which classifies pixels into one of ten classes, as the zones raster. Our values raster will be [Normalized difference vegetation index (NDVI)](https://en.wikipedia.org/wiki/Normalized_difference_vegetation_index), derived from [Sentinel-2 Level 2-A](https://planetarycomputer.microsoft.com/dataset/sentinel-2-l2a).

---

In [1]:
1+1

2

In [4]:
import numpy as np

import planetary_computer
import pystac_client
import stackstac
import rasterio.features

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

from xrspatial.multispectral import true_color

from xrspatial.classify import equal_interval
from xrspatial.zonal import stats as zonal_stats
from xrspatial.zonal import crosstab as zonal_crosstab

### Preparation: Create a local Dask cluster

We'll make a local Dask cluster to process the data in parallel.

In [2]:
from dask.distributed import Client

client = Client()
print(f"/proxy/{client.scheduler_info()['services']['dashboard']}/status")

/proxy/8787/status


To follow the progress of your computation, you can [access the Dask Dashboard](https://planetarycomputer.microsoft.com/docs/quickstarts/scale-with-dask/#Open-the-dashboard) at the URL from the previous cell's output.

### Load and coregister data

The area of interest covers Lake Bridgeport, Texas, USA. We'll use `pystac-client` to find items covering that area, and `stackstac` to load the items into a `DataArray`.

In [27]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1/",
    modifier=planetary_computer.sign_inplace,
)

bounds = (-98.00080760573508, 32.99921674609716, -96.9991860639418, 34.000729644613706)

landcover_search = catalog.search(collections=["io-lulc"], bbox=bounds)
landcover_items = landcover_search.item_collection()

landcover_data = (
    stackstac.stack(
        landcover_items,
        epsg=3857,
        bounds_latlon=bounds,
        dtype="int64",
        rescale=False,
        fill_value=0,
        chunksize=2048,
        resolution=100,
    )
    .pipe(stackstac.mosaic, nodata=0)
    .squeeze()
    .rename("Landcover")
    .persist()
)
landcover_data

<xarray.DataArray 'Landcover' (y: 1338, x: 1115)> Size: 12MB
dask.array<getitem, shape=(1338, 1115), dtype=int64, chunksize=(1338, 1115), chunktype=numpy.ndarray>
Coordinates:
  * y                  (y) float64 11kB 4.029e+06 4.029e+06 ... 3.895e+06
  * x                  (x) float64 9kB -1.091e+07 -1.091e+07 ... -1.08e+07
    band               <U4 16B 'data'
    label:description  <U4 16B 'lulc'
    end_datetime       <U20 80B '2021-01-01T00:00:00Z'
    start_datetime     <U20 80B '2020-01-01T00:00:00Z'
    label:properties   object 8B None
    label:type         <U6 24B 'raster'
    label:classes      object 8B {'name': '', 'classes': ['nodata', 'water', ...
    raster:bands       object 8B {'nodata': 0, 'spatial_resolution': 10}
    epsg               int64 8B 3857
Attributes:
    spec:        RasterSpec(epsg=3857, bounds=(-10909400, 3895100, -10797900,...
    crs:         epsg:3857
    transform:   | 100.00, 0.00,-10909400.00|\n| 0.00,-100.00, 4028900.00|\n|...
    resolution:  100

The landcover data includes some class labels, which we'll use to label the outputs later on.

In [28]:
landcover_labels = dict(
    enumerate(landcover_data.coords["label:classes"].item()["classes"])
)
landcover_labels

{0: 'nodata',
 1: 'water',
 2: 'trees',
 3: 'grass',
 4: 'flooded veg',
 5: 'crops',
 6: 'scrub',
 7: 'built area',
 8: 'bare',
 9: 'snow/ice',
 10: 'clouds'}

Finally, the landcover COGs also include a colormap.

In [ ]:
class_count = len(landcover_labels)

with rasterio.open(landcover_items[0].assets["data"].href) as src:
    landcover_colormap_def = src.colormap(1)  # get metadata colormap for band 1
    landcover_colormap = [
        np.array(landcover_colormap_def[i]) / 255 for i in range(class_count)
    ]

landcover_cmap = ListedColormap(landcover_colormap)

#### Sentinel data

Now we'll do the same thing, only for `sentinel-2-l2a` items from July 2020. We'll also try to select some not-too cloudy items and mosaic over time to remove as many clouds as possible.

In [6]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1/",
    modifier=planetary_computer.sign_inplace,
)

bounds = (-98.00080760573508, 32.99921674609716, -96.9991860639418, 34.000729644613706)

In [7]:
sentinel_search = catalog.search(
    collections=["sentinel-2-l2a"],
    bbox=bounds,
    datetime="2020-07-01/2020-07-30",
    query={
        "eo:cloud_cover": {
            "lt": 10,
        }
    },
)

sentinel_items = sentinel_search.item_collection()

We'll need the zones and values rasters to be in the same CRS and resolution, so we'll make sure to pass those values while we're loading the data with `stackstac` (see [Coregistration](coregistration.ipynb) for more).

In [9]:
sentinel_items

In [8]:
sentinel_data = (
    (
        stackstac.stack(
            sentinel_items,
            resolution=100,
            epsg=3857,
            bounds=bounds,
            assets=["B02", "B03", "B04", "B08"],  # blue, green, red, nir
            chunksize=2048,
        )
        .assign_coords(band=lambda x: x.common_name.rename("band"))  # use common names
        .where(lambda x: x > 0, other=np.nan)  # Sentinels uses 0 as nodata
    )
    .median(dim="time", keep_attrs=True)
    .persist()
)
sentinel_data

AttributeError: 'DataArray' object has no attribute 'common_name'

Create a true color image for the Sentinel data:

In [12]:
import odc.stac

In [14]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1/",
    modifier=planetary_computer.sign_inplace,
)

bounds = (-98.00080760573508, 32.99921674609716, -96.9991860639418, 34.000729644613706)

sentinel_search = catalog.search(
    collections=["sentinel-2-l2a"],
    bbox=bounds,
    datetime="2020-07-01/2020-07-30",
    query={
        "eo:cloud_cover": {
            "lt": 10,
        }
    },
)

# sentinel_items = sentinel_search.item_collection()
fetched_items = list(sentinel_search.items())
# len(items), items[0] if items else None

data_chunks = {'time': 1, 'x': 4096, 'y': 4096}
assets=["B02", "B03", "B04", "B08"]

sentinel_data = odc.stac.stac_load(items=fetched_items, bands=assets, bbox=bounds, groupby='time', chunks=data_chunks)

In [16]:
sentinel_data

<xarray.Dataset> Size: 10GB
Dimensions:      (y: 11241, x: 9466, time: 6)
Coordinates:
  * y            (y) float64 90kB 3.764e+06 3.764e+06 ... 3.652e+06 3.652e+06
  * x            (x) float64 76kB 5.923e+05 5.923e+05 ... 6.869e+05 6.869e+05
  * time         (time) datetime64[ns] 48B 2020-07-03T17:09:01.024000 ... 202...
    spatial_ref  int32 4B 32614
Data variables:
    B02          (time, y, x) float32 3GB dask.array<chunksize=(1, 4096, 4096), meta=np.ndarray>
    B03          (time, y, x) float32 3GB dask.array<chunksize=(1, 4096, 4096), meta=np.ndarray>
    B04          (time, y, x) float32 3GB dask.array<chunksize=(1, 4096, 4096), meta=np.ndarray>
    B08          (time, y, x) float32 3GB dask.array<chunksize=(1, 4096, 4096), meta=np.ndarray>

In [30]:
# Reduce time dimension so each band is 2D (y, x)
sentinel_data = sentinel_data.median(dim="time", keep_attrs=True).persist()

sentinel_img = true_color(
    sentinel_data["B04"],
    sentinel_data["B03"],
    sentinel_data["B02"],
    c=30,
    th=0.075,
    name="True color (Sentinel)",
)

KeyboardInterrupt: 

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(18, 8))

landcover_data.plot.imshow(ax=ax1, cmap=landcover_cmap)
sentinel_img.plot.imshow(ax=ax2)
plt.tight_layout()

<Figure size 1800x800 with 3 Axes>

As a final bit of setup, we'll compute NDVI for the sentinel-2 scene. This will be our values raster when we compute zonal statistics later on.

In [24]:
ndvi = ((sentinel_data["B08"] - sentinel_data["B04"]) / (sentinel_data["B08"] + sentinel_data["B04"])).persist()
ndvi

/home/ben/miniconda3/envs/forestHealth/lib/python3.14/site-packages/dask/_task_spec.py:759: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)


KeyboardInterrupt: 

### Compute zonal statistics

Now that we have our zones raster (the landcover DataArray) and our values raster (NDVI derived from the Sentinel composite), we're ready to compute zonal stats.

We'll use [xrspatial.zonal.stats](https://xarray-spatial.readthedocs.io/en/latest/reference/_autosummary/xrspatial.zonal.stats.html) to find a few summary statistics for each landcover class.

In [10]:
quantile_stats = (
    zonal_stats(
        zones=landcover_data,
        values=ndvi,
        stats_funcs=["mean", "max", "min", "count"],
    )
    .compute()
    .set_index("zone")
    .rename(landcover_labels)
)
quantile_stats.style.background_gradient()

NameError: name 'landcover_data' is not defined

Unsurprisingly, landcover classes like "trees" and "grass" have relatively high NDVI values. Classes like "water" and "bare" are relatively low.

### Compute zonal cross-tabulation statistics

[xrspatial.zonal.crosstab](https://xarray-spatial.readthedocs.io/en/latest/reference/_autosummary/xrspatial.zonal.crosstab.html) calculates cross-tabulated areas between the zone raster and value raster. This function requires a 2D zone raster and a categorical value raster of 2D or 3D data. 

The `crosstab` function calculates different cross-tabulation statistics. It has an `agg` parameter to define which aggregation method to use. It returns a DataFrame where each row represents a zone from the zone raster and each column represents a category from the value raster. 

This example uses the NASADEM elevation zones as its zone raster and the Esri Land Cover data as its categorical value raster. The resulting DataFrame will show the percentage of each land cover category for each of the five elevation zones.

In [ ]:
nasadem_search = catalog.search(collections=["nasadem"], bbox=bounds)
nasadem_items = nasadem_search.item_collection()

nasadem_data = (
    stackstac.stack(
        nasadem_items,
        epsg=landcover_data.spec.epsg,
        resolution=landcover_data.resolution,
        bounds=landcover_data.spec.bounds,
        chunksize=2048,
    )
    .pipe(stackstac.mosaic)
    .squeeze()
    .persist()
).rename("Elevation (NASADEM)")
nasadem_data

<xarray.DataArray 'Elevation (NASADEM)' (y: 1338, x: 1115)>
dask.array<getitem, shape=(1338, 1115), dtype=float64, chunksize=(1338, 1115), chunktype=numpy.ndarray>
Coordinates:
    band        <U9 'elevation'
  * x           (x) float64 -1.091e+07 -1.091e+07 ... -1.08e+07 -1.08e+07
  * y           (y) float64 4.029e+06 4.029e+06 ... 3.895e+06 3.895e+06
    proj:epsg   int64 4326
    proj:shape  object {3601}
    title       <U9 'Elevation'
    epsg        int64 3857
Attributes:
    spec:        RasterSpec(epsg=3857, bounds=(-10909400, 3895100, -10797900,...
    crs:         epsg:3857
    transform:   | 100.00, 0.00,-10909400.00|\n| 0.00,-100.00, 4028900.00|\n|...
    resolution:  100

Now let's discretize it into five zones.

In [ ]:
num_zones = 5
elevation_cmap = ListedColormap(plt.get_cmap("Set1").colors[:num_zones])

quantile_zones = equal_interval(
    nasadem_data, k=num_zones, name="Zones (Classified Elevation - NASADEM)"
)
fig, ax = plt.subplots(figsize=(8, 8))
img = quantile_zones.plot.imshow(cmap=elevation_cmap, ax=ax)
img.colorbar.set_ticks([0.4, 1.2, 2.0, 2.8, 3.6])
img.colorbar.set_ticklabels(range(5))

ax.set_axis_off()

<Figure size 800x800 with 2 Axes>

Finally, calculate the cross-tabulation statistics and display a table demonstrating how the land cover categories are distributed over each of the five elevation zones. Set values for `zone_ids` and `cat_ids` to indicate the zones and the categories of interest. In this example, we'll use all the available zones and categories.

In [ ]:
crosstab = (
    zonal_crosstab(
        zones=quantile_zones,
        values=landcover_data,
        agg="percentage",
    )
    .rename(columns=landcover_labels)
    .compute()
    .set_index("zone")
)

crosstab.style.background_gradient()

,water,trees,grass,flooded veg,crops,scrub,built area,bare
zone,,,,,,,,
0.000000,19.270796,16.616853,10.577962,0.395211,7.421678,7.088585,38.439863,0.189053
1.000000,2.027249,23.448931,22.147768,0.109607,20.129374,13.828654,17.774741,0.533675
2.000000,2.075033,20.599665,23.654604,0.090979,16.611000,27.911911,8.759015,0.297792
3.000000,0.596741,27.804037,21.278863,0.029319,14.013572,28.321761,7.864891,0.090816
4.000000,0.216216,34.702703,19.106518,0.003180,6.206677,32.282989,7.033386,0.448331


### Next steps: analyze different datasets

The [Planetary Computer Data Catalog](https://planetarycomputer.microsoft.com/catalog) includes petabytes of environmental monitoring data. All data sets are available in consistent, analysis-ready formats. You can access them through APIs as well as directly via [Azure Storage](https://docs.microsoft.com/en-us/azure/storage/). 

Try using [xrspatial.zonal.stats](https://xarray-spatial.readthedocs.io/en/latest/reference/_autosummary/xrspatial.zonal.stats.html) and [xrspatial.zonal.crosstab](https://xarray-spatial.readthedocs.io/en/latest/reference/_autosummary/xrspatial.zonal.crosstab.html) with different datasets from the [Data Catalog](https://planetarycomputer.microsoft.com/catalog). For example, try using the land cover categories from the [Esri 10m Land Cover](https://www.arcgis.com/home/item.html?id=d6642f8a4f6d4685a24ae2dc0c73d4ac) dataset as zonal raster. Or try using the [Map of Biodiversity Importance (MoBI)](https://planetarycomputer.microsoft.com/dataset/mobi) dataset as a values raster.

There are also [other zonal functions in xarray spatial](https://xarray-spatial.readthedocs.io/en/latest/reference/zonal.html) to use with your datasets.